In [2]:
#open pandas DF 
import pandas as pd
Puerto_Rico_data= pd.read_csv("../Raw_Data/NCRMP_Puerto_Rico_raw.csv")
print(Puerto_Rico_data.columns)

Index(['time', 'latitude', 'longitude', 'REGION', 'REGION_DESCRIPTION',
       'PRIMARY_SAMPLE_UNIT', 'STATION_NR', 'YEAR', 'MONTH', 'DAY', 'Date_UTC',
       'HABITAT_CD', 'HABITAT_TYPE', 'STRAT', 'STRAT_Description',
       'RUGOSITY_CD', 'WTD_RUG', 'MEAN_RUG', 'MAPGRID_NR', 'SUB_REGION_NAME',
       'SUB_REGION_NAME_DESCRIPTION', 'SUB_REGION_NR', 'ZONE_NAME', 'ZONE_NR',
       'MPA_NAME', 'MPA_NR', 'ADMIN', 'Administration_Description', 'PROT',
       'DEPTH_STRAT', 'DEPTH_STRAT_DESCRIPTION', 'MIN_DEPTH', 'MAX_DEPTH',
       'METERS_COMPLETED', 'SPECIES_CD', 'SPECIES_NAME', 'N', 'JUV',
       'MAX_DIAMETER', 'PERP_DIAMETER', 'HEIGHT', 'OLD_MORT', 'RECENT_MORT',
       'BLEACH_CONDITION', 'DISEASE', 'accession_url'],
      dtype='object')


/var/folders/17/9kvd528n47j282gcln4v5zvc0000gp/T/ipykernel_1477/1153998520.py:3: DtypeWarning: Columns (1,2,17,31,32,33,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  Puerto_Rico_data= pd.read_csv("../Raw_Data/NCRMP_Puerto_Rico_raw.csv")


In [3]:
# isolating Genus for counting
Puerto_Rico_data = Puerto_Rico_data[["SPECIES_CD","SPECIES_NAME","DISEASE"]]
Puerto_Rico_data = Puerto_Rico_data.dropna(how='all').reset_index(drop=True)
Puerto_Rico_data

,SPECIES_CD,SPECIES_NAME,DISEASE
0,ACR CERV,Acropora cervicornis,NaN
1,ACR CERV,Acropora cervicornis,NaN
2,ACR CERV,Acropora cervicornis,NaN
3,ACR CERV,Acropora cervicornis,NaN
4,ACR CERV,Acropora cervicornis,NaN
...,...,...,...
41450,ACR PALM,Acropora palmata,NaN
41451,ACR CERV,Acropora cervicornis,NaN
41452,ACR CERV,Acropora cervicornis,NaN
41453,ACR CERV,Acropora cervicornis,NaN


In [4]:
# select Disease column and setting empty cells to healthy 
Puerto_Rico_data['DISEASE'][Puerto_Rico_data['DISEASE'].isnull()] = "Unknown"
Puerto_Rico_data

,SPECIES_CD,SPECIES_NAME,DISEASE
0,ACR CERV,Acropora cervicornis,Unknown
1,ACR CERV,Acropora cervicornis,Unknown
2,ACR CERV,Acropora cervicornis,Unknown
3,ACR CERV,Acropora cervicornis,Unknown
4,ACR CERV,Acropora cervicornis,Unknown
...,...,...,...
41450,ACR PALM,Acropora palmata,Unknown
41451,ACR CERV,Acropora cervicornis,Unknown
41452,ACR CERV,Acropora cervicornis,Unknown
41453,ACR CERV,Acropora cervicornis,Unknown


In [5]:
#Count of corals by genus and Disease column 
Grouped_Puerto_Rico_data = Puerto_Rico_data.groupby(['SPECIES_NAME','DISEASE']).agg(Disease_count = ("DISEASE", "count")).reset_index()

Grouped_Puerto_Rico_data

,SPECIES_NAME,DISEASE,Disease_count
0,Acropora cervicornis,A,76
1,Acropora cervicornis,P,7
2,Acropora cervicornis,Unknown,572
3,Acropora palmata,A,27
4,Acropora palmata,P,2
...,...,...,...
159,Tubastraea coccinea,A,4
160,Tubastraea coccinea,Unknown,269
161,Undaria spp,A,23
162,Undaria spp,P,2


In [10]:
from numpy import isfinite
from math import isnan
disease_df = Puerto_Rico_data
unique_diseases = set(disease_df["DISEASE"].unique())

df = Puerto_Rico_data
unique_species = set(df["SPECIES_NAME"].unique())
unique_species = list(unique_species)
print(unique_species)
print(unique_diseases)

disease_table = pd.DataFrame(0, index=unique_species,columns=unique_diseases)
for species_name in unique_species:
    for disease in unique_diseases: 
        disease_subtable = df.loc[(df['SPECIES_NAME'] == species_name) & (df['DISEASE'] == disease)]
        empty = bool(disease_subtable.empty)
        if not empty:
            count = disease_subtable['DISEASE'].count()
        
            disease_table.loc[species_name, disease] = count
disease_table.sort_index()

disease_table = disease_table.rename(columns ={'A':'healthy_count', 'P':'total_diseased_count', 'Unknown':'unknown_count'})
disease_table

disease_table["total_diseased_percent"] = disease_table["total_diseased_count"]/(disease_table["healthy_count"]+ disease_table["total_diseased_count"])

disease_table = disease_table.drop(columns=['F','S'])
disease_table = disease_table.rename_axis('species')
disease_table

['Agaricia agaricites', 'Porites astreoides', 'Orbicella annularis species complex', 'Madracis spp', 'Agaricia humilis', 'Stephanocoenia intersepta', 'Acropora prolifera', 'Madracis pharensis', 'Porites porites', 'Favia fragum', 'Siderastrea spp', 'Orbicella faveolata', 'Agaricia lamarcki', 'Undaria spp', 'Meandrina spp', 'Acropora palmata', 'Meandrina meandrites', 'Isophyllastrea rigida', 'Siderastrea siderea', 'Mycetophyllia danaana', 'Madracis senaria', 'Mussa angulosa', 'Agaricia tenuifolia', 'Agaricia grahamae', 'Agaricia fragilis', 'Orbicella franksi', 'Pseudodiploria clivosa', 'Scolymia cubensis', 'Solenastrea bournoni', 'Mycetophyllia ferox', 'Orbicella spp', 'Madracis carmabi', 'Porites branneri', 'Eusmilia fastigiata', 'Porites furcata', 'Meandrina danae', 'Dendrogyra cylindrus', 'Agaricia spp', 'Helioceris cucullata', 'Meandrina jacksoni', 'Porites divaricata', 'Scolymia spp', 'Tubastraea coccinea', 'Diploria labyrinthiformis', 'Madracis formosa', 'Oculina diffusa', 'Dichoco

,unknown_count,healthy_count,total_diseased_count,total_diseased_percent
species,,,,
Agaricia agaricites,286,1547,11,0.007060
Porites astreoides,121,4619,49,0.010497
Orbicella annularis species complex,319,1,1,0.500000
Madracis spp,427,7,0,0.000000
Agaricia humilis,521,320,2,0.006211
...,...,...,...,...
Pseudodiploria strigosa,266,1159,29,0.024411
Mycetophyllia aliciae,434,11,0,0.000000
Orbicella annularis,493,346,12,0.033520


In [11]:
#finish cleaning up
#add stuyd info to meta data (https://www.ncei.noaa.gov/erddap/tabledap/CRCP_Coral_Demographic_Survey_Puerto_Rico.html)

In [12]:
disease_table.to_csv("../Processed_Data/NCRMP_Puerto_Rico_corals.csv",index_label="species_name")